# (05-05): Dá upload aos posts enquanto os processa, remove duplicas, otimizado na recuperação de contexto 

In [1]:
from langchain_core.runnables import  RunnablePassthrough
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.graphs.graph_document import GraphDocument, Node, Relationship
from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
from langchain_ollama import ChatOllama
import hashlib
from langchain_core.prompts import SystemMessagePromptTemplate, PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.docstore.document import Document
import json
import uuid
from sentence_transformers import SentenceTransformer, util

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
graph = Neo4jGraph()

C:\Users\jocke\AppData\Local\Temp\ipykernel_63720\267675202.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


# Extrair comentários de uma thread

Scraper 

In [3]:
import praw
import re

class RedditThreadScraper:
    def __init__(self, client_id, client_secret, user_agent):
        self.reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)
        self.comment_tree = {}
        self.root_id = None

    def limpar_texto(self, text):
                """
                Cleans text from social media comments, converting citation markers and retaining all content.
                Converts '>' to '**Quoting**' and '\n\n' to '**End of Quote**' for citations, then removes
                unwanted characters and normalizes whitespace. Handles multiple citations and preserves
                all text.
                
                Args:
                    text (str): Raw text from social media comments.
                
                Returns:
                    str: Cleaned text with converted citation markers and all content preserved.
                """
                # Step 1: Split text by '\n\n' to identify citation and non-citation parts
                parts = text.strip().split('\n\n')
                converted_parts = []
                
                for part in parts:
                    part = part.strip()
                    if part.startswith('>'):
                        # Step 2: Handle citation - convert '>' to '*Quoting*'
                        citation_content = part[1:].strip()  # Remove leading '>'
                        # Add '*Quoting*' and '*End of Quote*' explicitly
                        converted_parts.append(f"**Quoting** {citation_content} **End of Quote**")
                    else:
                        # Step 3: Handle non-citation text
                        converted_parts.append(part)
                
                # Step 4: Join all parts to process as a whole
                text = '\n\n'.join(converted_parts)
                
                # Step 5: Clean the text
                # Remove unwanted characters (e.g., emojis, excessive punctuation, hashtags, mentions)
                # Preserve '*Quoting*' and '*End of Quote*' by excluding '*' from cleaning
                text = re.sub(r'[^\w\s.,!?*]', '', text)
                
                # Normalize whitespace (replace multiple spaces/tabs with single space)
                # Do this carefully to avoid collapsing '\n\n'
                lines = text.split('\n\n')
                cleaned_lines = []
                for line in lines:
                    # Normalize spaces within each line
                    cleaned_line = re.sub(r'\s+', ' ', line).strip()
                    if cleaned_line:
                        cleaned_lines.append(cleaned_line)
                
                # Step 6: Join with '\n\n' to maintain separation
                return '\n\n'.join(cleaned_lines)

    def build_comment_tree(self, thread_url):
        try:
            submission = self.reddit.submission(url=thread_url)
            titulo_op = self.limpar_texto(submission.title)
            submission.comments.replace_more(limit=0)
            self.comment_tree = {}
            self.root_id = submission.id
            self.comment_tree[self.root_id] = {
                "id": self.root_id,
                "title": titulo_op,
                "tags": [submission.link_flair_text] if submission.link_flair_text else [],  # Tags only for OP
                "text": self.limpar_texto(submission.selftext),
                "author": submission.author.name if submission.author else "Unknown",
                "parent_id": None,
                "children": []
            }
            for comment in submission.comments.list():
                if comment.author != "AutoModerator":
                    comment_id = comment.id
                    parent_id = comment.parent_id.split("_")[-1]
                    self.comment_tree[comment_id] = {
                        "id": comment_id,
                        "text": self.limpar_texto(comment.body),
                        "author": comment.author.name if comment.author else "Unknown",
                        "parent_id": parent_id,
                        "children": []  # No "tags" field for comments
                    }
                    if parent_id in self.comment_tree:
                        self.comment_tree[parent_id]["children"].append(comment_id)
            return True
        except Exception as e:
            print(f"Erro ao construir a árvore de comentários: {e}")
            self.comment_tree = {}
            self.root_id = None
            return False

    def get_comment_tree(self):
        return self.comment_tree

    def get_root_id(self):
        return self.root_id

    def print_tree(self, node_id=None, level=0):
        if not self.comment_tree:
            print("Árvore de comentários vazia.")
            return
        if node_id is None:
            node_id = self.root_id
        if node_id not in self.comment_tree:
            print(f"Nó {node_id} não encontrado na árvore.")
            return
        node = self.comment_tree[node_id]
        prefix = "➡" if level == 0 else " " * (level * 4) + "↳"
        # Only show tags for the root node (OP)
        if node_id == self.root_id:
            print(f"{prefix} {node['author']}: {node['text'][:100]} (Tags: {', '.join(node['tags'])})")
        else:
            print(f"{prefix} {node['author']}: {node['text'][:100]}")
        for child_id in node["children"]:
            self.print_tree(child_id, level + 1)

Scrape da thread

In [4]:
#from reddit_scrapper_class import *

scraper = RedditThreadScraper(
        client_id=os.getenv("REDDIT_CLIENT_ID"),
        client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
        user_agent="meu_bot_para_scraping"
    )
    
thread_url = "https://www.reddit.com/r/linux/comments/1klieoy/what_would_be_a_linux_like_desktop_setup/"

if scraper.build_comment_tree(thread_url):
    
    print(f"\nTítulo do OP: {scraper.comment_tree[scraper.root_id]['title']}\n")
    scraper.print_tree()




Título do OP: What would be a Linux like desktop setup?

➡ FryChy: removed (Tags: Discussion)
    ↳ nevyn28: Whatever you want it to be
        ↳ dudleydidwrong: This is the most stereotypical Linux answer. In this case, it is also the correct answer.

I remembe
    ↳ PilotKnob: The best part is, its your choice.

There are no wrong solutions.
    ↳ Bali10050: Tiling window managers
        ↳ ppp7032: that and gnome are the two most quintessential layouts imo
    ↳ high-tech-low-life: I would say it was a foolish or ignorant question. Lack of enforced uniformity is one of the hallmar
    ↳ Huge-Actuator-6504: If Im talking to someone whos clearly selfabsorbed in their own preconceptions about Linux that its 
    ↳ BigYoSpeck: I dont think theres any such thing as a Linux like desktop. Linux desktop distros have always just a
        ↳ FattyDrake: **Quoting** I dont think theres any such thing as a Linux like desktop. Linux desktop distros have a
            ↳ BigYoSpeck: Good artists 

In [ ]:
scraper.comment_tree

{'1kd2f1o': {'id': '1kd2f1o',
  'title': 'Mint and Des Wayland?',
  'tags': ['Desktop Environment / WM News'],
  'text': 'Hi guys,\n\nIm a Mint user, I know some people like it some people hate it but to me it just do what its meant to do. No bug, never had big issues, wellsupported by community.\n\nAs Im a lover of breaking and experimenting I was wondering if I could change my Mint DE with something more lightweighted.\n\nI like Cinnamon but I can see it slow performance a bit, expecially for gpu and gaming.\n\nIm trying Wayland experimental but I cannot see difference with Cinnamon and I read it does not increase pc performance Anyway it solved my laggy logitech Mx Master 3s for some reason.\n\nIve a full AMD Desktop pc R5 2600,RX6600XT 8GB, 16GB Ram, my main use is gaming and editing videophoto for fun, aaand breaking my distro as well.\n\nSo Should I try other DEs or change distro at all?\n\nMain goal Lightweight my distro as much as possible but still customizable and usable Not 

# Recuperação de contexto e criação de prompt

Prompt:

In [5]:
# Pedir a contextualizão de argumentos pode levar a tempos de processamento maiores (de 3 passou para 6 segundos)
#TODO: acresentar contexto para cada categorria max-neef (para melhorar resultados)
system_prompt = SystemMessagePromptTemplate.from_template("""
You are a system that extracts political arguments and their associated motivations from input text. Your goal is to return a dataset of clearly defined [Argument] and [Motivation] sections in **JSON format**. 

You will be given a political discussion thread with multiple comments. These comments are presented together as input, and may include original posts and replies. You should extract arguments and motivations based on the full context of the thread — not just the last comment.

                                                          
Each [Argument] must contain a single contextualized and coherent political claim or position. Each [Motivation] must explain a reason behind the argument and be associated with one or more of the following Max-Neef categories:
Subsistence, Protection, Affection, Understanding, Participation, Leisure, Creativity, Identity, Freedom.

You must preserve a clear link between each Argument and its corresponding Motivations by nesting motivations under their respective argument.

Here are a few examples of the expected structure:

[
  {{
    "argument": "Tax policy should be expanded to support economic growth",
    "motivations": [
      {{
        "description": "Wants economic stability for all",
        "max_neef_category": ["Subsistence"]
      }}
    ]
  }},
  {{
    "argument": "Climate change needs tighter restrictions to safeguard future generations",
    "motivations": [
      {{
        "description": "Wants to secure the future",
        "max_neef_category": ["Protection"]
      }},
      {{
        "description": "Desires collective action",
        "max_neef_category": ["Participation"]
      }}
    ]
  }},
  {{
    "argument": "Healthcare policies should focus on advancing social equity",
    "motivations": [
      {{
        "description": "Hopes to support the vulnerable",
        "max_neef_category": ["Affection"]
      }}
    ]
  }},
  {{
    "argument": "Gun control needs tighter restrictions to protect public safety",
    "motivations": [
      {{
        "description": "Fears threats to safety",
        "max_neef_category": ["Protection"]
      }}
    ]
  }},
  {{
    "argument": "Arts funding should prioritize fostering cultural development",
    "motivations": [
      {{
        "description": "Enjoys cultural enrichment",
        "max_neef_category": ["Leisure"]
      }},
      {{
        "description": "Wants innovative solutions",
        "max_neef_category": ["Creativity", "Understanding"]
      }}
    ]
  }}
]

Using this JSON format, extract arguments and motivations from the input text. Each argument must be politically relevant and distinct. Motivations must be logically aligned with the argument.
""")

human_prompt = HumanMessagePromptTemplate.from_template("""
You are given a sequence of political comments from a discussion thread. Use the entire thread as context.

Make sure to group Arguments with their respecting Motivations. Use the format shown above to extract the relevant information from the following input:

Thread:
                                                        
{input}

Extract arguments and motivations **only from the last comment**, and group each argument with its respective motivations.

                                                        
Make sure **each Argument has one or more associated Motivations**, and that the response uses the correct format. Do not include any commentary or explanations.

""")

chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])


Funções para recolher contexto (com um summarizer e otimizado para guardar as respostas/processar apenas comentários grandes)

In [6]:

summarizer = ChatOllama(model="llama3.1")

resumos_cache = {} # Para guardar os resumos dos posts anteriores

# Estimativa de tokens com base nas palavras
def estimar_tokens(texto: str) -> int:
    return int(len(texto.split()) * 1.3)  # aproximação razoável

# Função para buscar os pais de um post (até 3 níveis)
def get_parent_posts(post_id: str, max_levels: int = 3):
    query = f"""
    MATCH (child:Post {{id: $post_id}})-[:RESPONDS_TO*1..{max_levels}]->(parent:Post)
    WHERE toLower(parent.text) <> '[removed]'
    RETURN parent.text AS text, parent.id AS id
    ORDER BY size(parent.text) DESC
    """
    results = graph.query(query, params={"post_id": post_id})
    return [{"id": row["id"], "text": row["text"]} for row in results]

# Função para buscar o texto do post atual
def get_post_text(post_id: str):
    query = """
    MATCH (p:Post {id: $post_id})
    RETURN p.text AS text
    """
    result = graph.query(query, params={"post_id": post_id})
    return result[0]["text"] if result else None

# Função para resumir texto com LLM e verificação de tamanho e cache
def resumir_texto(texto: str, token_threshold: int = 150) -> str:
    if texto in resumos_cache:
        return resumos_cache[texto]
    
    if estimar_tokens(texto) <= token_threshold:
        resumo = texto.strip()
    else:
        prompt = f"""
        Summarize the following post in a concise and informative way. If you refer to the author, make sure you refer to them as 'an author of previous posts'. Only keep what is essential to understand the point made, and return only the summary:

        \"\"\"{texto}\"\"\"
        """
        resumo = summarizer.invoke(prompt).content.strip()
    
    resumos_cache[texto] = resumo
    return resumo

# Função principal para construir o contexto sumarizado a partir de posts anteriores
def get_context(post_id: str, max_pais: int = 3, token_threshold: int = 200):
    pais = get_parent_posts(post_id, max_levels=max_pais)
    post_filho = get_post_text(post_id)

    if not post_filho:
        raise ValueError("Post filho não encontrado.")

    contexto_resumido = []
    for p in pais[:max_pais]:
        resumo = resumir_texto(p["text"], token_threshold=token_threshold)
        contexto_resumido.append(f"*Context:* {resumo}")

    contexto_final = "\n".join(contexto_resumido)
    return contexto_final
    




In [ ]:
# Exemplo de utilização

post_id_input = "mlk6gmm"  
context = get_context(post_id_input)
print(context)

# Preparar loop de processamento

Funcao para converter output json para graph document e invocacao do llm pelo ollama

In [7]:
def json_para_graph_document(data: dict, input_text: str) -> GraphDocument:
    graph_documents = []
    
    nodes = []
    relationships = []
    
    for entry in data.get("arguments", []):
        # Cria o nó do argumento
        argument_id = str(uuid.uuid4())
        argument_node = Node(
            id=argument_id,
            type="Argument",
            properties={"description": entry["argument"]}
        )
        nodes.append(argument_node)
        
        for motivation in entry.get("motivations", []):
            # Cria o nó da motivação
            motivation_id = str(uuid.uuid4())
            motivation_node = Node(
                id=motivation_id,
                type="Motivation",
                properties={
                    "description": motivation["description"],
                    "max_neef_category": motivation["max_neef_category"]
                }
            )
            nodes.append(motivation_node)
            
            # Cria a relação entre motivação e argumento
            relationship = Relationship(
                source=motivation_node,
                target=argument_node,
                type="supports",
                properties={}
            )

            relationships.append(relationship)

    # Documento de origem (comentário, post, etc.)
    doc = Document(page_content=input_text)

    graph_doc = GraphDocument(
        nodes=nodes,
        relationships=relationships,
        source=doc
    )
    
    graph_documents.append(graph_doc)
    return graph_documents

In [8]:
llm = ChatOllama(model="llama3.1", temperature=0, format="json")

def extract_nodes(input_text):
    messages = chat_prompt.format_messages(input=input_text)
    response = llm.invoke(messages)
    
    return json.loads(response.content)   

comment_tree = scraper.comment_tree

Recolha do titulo do topico (util para anexar argumentos aos posts mais à frente) e func para dar upload ao nó do post

In [9]:

def get_original_post_title(comment_tree):
    '''Helper function to find the original post's title, optimized for parent node being first'''
    # Check if the first node in comment_tree is the original post
    first_post = next(iter(comment_tree.values()), None)
    if first_post and first_post.get("parent_id") is None and "title" in first_post:
        return first_post["title"]


topic_title = get_original_post_title(comment_tree=comment_tree)


def upload_post(comment_info, topic_title):
    ''' Função para dar upload aos posts. Caso seja o post original cria tambem um topico de discussao'''

    if "title" in comment_info:  # Original Post

        # Create/Update the Post node for OP
        graph.query("""
            MERGE (p:Post {id: $id})
            SET p.text = $text, p.author = $author, p.topic_title = $topic_title
            RETURN p
            """, 
            {"id": comment_info["id"], "text": comment_info["text"], "author": comment_info["author"], "topic_title":  topic_title}
        )
        
        # Create/Update the DiscussionTopic node with tags and link it to the Post
        graph.query("""
            MERGE (dt:DiscussionTopic {title: $title})
            SET dt.tags = $tags  
            WITH dt
            MATCH (p:Post {id: $post_id})
            MERGE (dt)-[:DISCUSSES]->(p)
            """,
            {"title": comment_info["title"], "post_id": comment_info["id"], "tags": comment_info["tags"]  }
        )
    
    else : 
        #posts filhos

        parent_id = comment_info['parent_id']
        comment_tree.get(parent_id)
        graph.query("""
            MERGE (p:Post {id: $id})
            SET p.text = $text, p.author = $author, p.topic_title = $topic_title
            RETURN p
            """, 
            {"id": comment_info["id"], "text": comment_info["text"], "author": comment_info["author"], "topic_title":  topic_title}
        )
    
        #verificar relações 
        graph.query("""
                    MATCH (child:Post {id: $child_id}), (parent:Post {id: $parent_id})
                    MERGE (child)-[:RESPONDS_TO]->(parent)
                    """, 
                    {"child_id": comment_info["id"], "parent_id": comment_info["parent_id"]}
                )


Preparar verificacao de duplicados com embeddings


In [10]:
# Carregar modelo de embeddings uma vez
modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')

def filtrar_argumentos_unicos(raw_format: dict, argumentos_processados: list[str], threshold: float = 0.90) -> dict:
    """
    Filtra os argumentos duplicados com base em embeddings. Atualiza raw_format.
    """
    argumentos_validos = []

    for arg_data in raw_format.get("arguments", []):
        argumento = arg_data.get("argument", "").strip()

        if not argumento:
            continue

        if argumentos_processados:
            embedding_novo = modelo_embeddings.encode(argumento, convert_to_tensor=True)
            embeddings_existentes = modelo_embeddings.encode(argumentos_processados, convert_to_tensor=True)
            similaridades = util.cos_sim(embedding_novo, embeddings_existentes)
            if any(sim > threshold for sim in similaridades[0]):
                print(f"⚠️ Argumento ignorado por ser duplicado: {argumento[:80]}...")
                continue

        argumentos_validos.append(arg_data)
        argumentos_processados.append(argumento)

    raw_format["arguments"] = argumentos_validos
    return raw_format

## Loop de processamento principal

In [11]:


graph_documents = []
n_comments = len(scraper.comment_tree.values())
count = 1

for comment_info in scraper.comment_tree.values():
    argumentos_processados = []
    print(f"Processing comment {count}/{n_comments}")
    count += 1
    
    #upload de nó post para a bdd
    upload_post(comment_info,topic_title)
    
    #Preparacao para Extração de argumento/motivacao
    comment = comment_info.get("text")
    context = get_context(comment_info.get("id"))
    input = context + "\n" +comment

    #Extracao dos nodes
    raw_format = extract_nodes(input)
    
    #Remoção de duplicados
    raw_format =filtrar_argumentos_unicos(raw_format, argumentos_processados)

    #Passar de json para GraphDocument (para facilitar o upload)
    temp_doc = json_para_graph_document(raw_format, comment)
    
    #Adicionar o post id aos metadados dos nodes
    temp_doc[0].source.metadata['post_id'] = comment_info.get("id") 

    graph_documents.extend(temp_doc)


Processing comment 1/19
Processing comment 2/19


BufferError: Existing exports of data: object cannot be re-sized

In [28]:
for doc in graph_documents:
    print(doc.nodes)


[]
[Node(id='c5dfb4bc-ac40-4c36-82dc-fa3eeead13b1', type='Argument', properties={'description': 'The DE will have a minimal impact on performance'}), Node(id='01955006-da73-433f-bd4b-be9a85a2d671', type='Motivation', properties={'description': 'Wants to minimize effort', 'max_neef_category': ['Leisure']}), Node(id='a8386a49-fc40-4277-bb1f-51a04cb362cb', type='Motivation', properties={'description': 'Fears unnecessary complexity', 'max_neef_category': ['Protection']})]
[Node(id='bb672f57-bf08-4d0a-8134-da1e33e73148', type='Argument', properties={'description': 'The current gaming performance tweaks have been effective for over a year'}), Node(id='e23753f9-87bf-4d63-9040-dc329e084b9c', type='Motivation', properties={'description': 'Wants to maintain optimal gaming experience', 'max_neef_category': ['Leisure']}), Node(id='f739376b-072a-466c-8c85-739a9621c951', type='Argument', properties={'description': 'The system has been running beautifully on my PC'}), Node(id='49210154-e376-464e-99e7

# Exportar nodes e relações

In [30]:
# adicionar nodes origem

# Upload do grafo com argumentos e motivações
graph.add_graph_documents(graph_documents=graph_documents, include_source=False)

# Criar ou obter o nó do tópico e conectar posts a ele
for doc in graph_documents:
    post_id = doc.source.metadata.get('post_id')
    # Conectar argumentos ao post
    for node in doc.nodes:
        if "Argument" in node.type:
            graph.query(
                """
                MATCH (p:Post {id: $post_id}), (n:Argument {id: $node_id})
                MERGE (p)-[:STATED]->(n)
                """,
                {"post_id": post_id, "node_id": node.id}
            )





## Debugging

### Parte de verificacao de embeddings

In [ ]:

temp = extract_nodes('''**Quoting** Im trying Wayland experimental but I cannot see difference with Cinnamon and I read it does not increase pc performance Anyway it solved my laggy logitech Mx Master 3s for some reason. **End of Quote**

I mean.. Ive definitely seen some improvement in games Euro Truck Sim 2 on Wayland. Not sure why, but I have. This is pretty much supported by recent benchmarks from Phoronix.

Cinnamon is, usually, from the benchmarks Ive seen, a little behind both Gnome and KDE Plasma. My guess is that is that its because Muffin is forked from Mutter, and is lacking behind in any improvements made to Mutter.

Realistically speaking, youre unlikely to see any real difference, except in edge cases, or unless youre running into memory constrained scenarios.

For reference, I have an older machine with only 8GB of RAM running Baldurs Gate 3 8GB recommended minimum superbly without issues on Gnome 48.''')

In [ ]:

duplicado = {
    "argument": "I like sunny days",
    "motivations": [
        {"description": "Same motivation again to test duplication",
         "max_neef_category": ["Understanding"]}
    ]
}

# Adicionar à lista de argumentos
temp["arguments"].append(duplicado)

argumentos_processados = []

filtrar_argumentos_unicos(temp, argumentos_processados)